# Topic Modeling (Prepare)

On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophisticated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifying common themes in customer reviews
* Discovering the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* Part 0: Warm-Up
* Part 1: Describe how an LDA Model works
* Part 2: Estimate a LDA Model with Gensim
* Part 3: Interpret LDA results & Select the appropriate number of topics

# Part 0: Warm-Up
How do we do a grid search? 

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Load training data
newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'))

# Load testing data
newsgroups_test = fetch_20newsgroups(subset='test', 
                                     remove=('headers', 'footers', 'quotes'))

print(f'Training Samples: {len(newsgroups_train.data)}')
print(f'Testing Samples: {len(newsgroups_test.data)}')

Training Samples: 11314
Testing Samples: 7532


In [4]:
newsgroups_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
newsgroups_train['target_names']

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
newsgroups_train['data'][1000]

"Anybody seen mouse cursor distortion running the Diamond 1024x768x256 driver?\nSorry, don't know the version of the driver (no indication in the menus) but it's a recently\ndelivered Gateway system.  Am going to try the latest drivers from Diamond BBS but wondered\nif anyone else had seen this.\n\npost or email"

### GridSearch on Just Classifier
* Fit the vectorizer and prepare BEFORE it goes into the gridsearch

In [7]:
# Instantiate vectorizer
vect = TfidfVectorizer()

# Transform the training data
X_train = vect.fit_transform(newsgroups_train['data'])
print(X_train.shape)

(11314, 101631)


In [8]:
params_1 = {
    'min_samples_leaf': [1, 2, 5, 10]
}

# Instantiate classifier
clf = RandomForestClassifier()

# GridSearch
gs1 = GridSearchCV(clf, params_1, cv=5, n_jobs=-1, verbose=1)
gs1.fit(X_train, newsgroups_train['target'])

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  4.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [9]:
gs1.best_score_

0.6536154404866977

In [10]:
gs1.best_params_

{'min_samples_leaf': 2}

In [11]:
test_sample = vect.transform(["The new york yankees are the best team in the region."])
test_sample.shape

(1, 101631)

In [12]:
gs1.predict(test_sample)[0]

9

In [13]:
newsgroups_train['target_names'][9]

'rec.sport.baseball'

### GridSearch with BOTH the Vectoizer & Classifier

In [15]:
from sklearn.pipeline import Pipeline

# 1. Create a pipeline with a vectorize and a classifier
# 2. Use Grid Search to optimize the entire pipeline
pipe = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', RandomForestClassifier(random_state=42))
    
])

params_2 = {
    'vect__stop_words': (None, 'english'),# , SPACY_STOP_WORDS, CUSTOM_STOP_WORDS),
    'vect__min_df': (2, 5),
    'clf__max_depth': (10, None)

}

gs2 = GridSearchCV(pipe, params_2, cv=5, n_jobs=-1, verbose=1)
gs2.fit(newsgroups_train['data'], newsgroups_train['target'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  4.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        no

In [16]:
gs2.best_score_

0.6607746264533867

In [17]:
gs2.best_params_

{'clf__max_depth': None, 'vect__min_df': 2, 'vect__stop_words': 'english'}

In [18]:
pred = gs2.predict(["The new york yankees are the best team in the region."])
pred

array([9])

In [19]:
newsgroups_train['target_names'][pred[0]]

'rec.sport.baseball'

Advantages to using GS with the Pipe:
* Allows us to make predictions on raw text increasing reproducibility. :)
* Allows us to tune the parameters of the vectorizer along side the classifier. :D 

# Part 1: Describe how an LDA Model works

[Your Guide to Latent Dirichlet Allocation](https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d)

[LDA Topic Modeling](https://lettier.com/projects/lda-topic-modeling/)

[Topic Modeling with Gensim](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)

In [20]:
# Download spacy model
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [21]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
df = pd.DataFrame({
    'content': newsgroups_train['data'],
    'target': newsgroups_train['target'],
    'target_names': [newsgroups_train['target_names'][i] for i in newsgroups_train['target']]
})
print(df.shape)

(11314, 3)


In [23]:
pd.set_option('display.max_colwidth', 0)
df.sample(3)

,content,target,target_names
8141,"I recently bought an apparantly complete Expansion Chassis by Mountain\nComputer Inc. It consists of a box with 8 Apple ][+ compatible slots,\npowersupply brick, interface card and ribbon cable to attach it to the computer\nto be expanded. There was also included a small card with empty sockets on top\nand pins on the bottom that looks like it would plug into the ][+ motherboard\nsomewhere after pulling a chip. There's an empty socket also on the interface\ncard and a short 16-pin DIP jumper like the ones used with ][+ language cards.\n \nThis technological marvel came with no docs and I haven't a clue as how to hook\nthis thing up. If anyone has docs and/or users disk of any sort for this I\ncould really use copies of them or at least some help.\n \nI need to know:\n \no How to orient the ribbon cable between the card and the chassis.\no How to attach the short cable from the motherboard to the card\n and if the small card is used.\no The purposes of the various jumper-pins on the card (it has more\n of those than my CMS SCSI card!)\n",12,sci.electronics
9323,"Seems that the Mile-Long Billboard and any other inflateble space\nobject/station or what ever have the same problems. (other than being a little\nbit different than the ""normal"" space ideas, such as trusses and shuttles)\n\nBut also dag and such.. Why not combine the discussion of how and fesibility to\nthe same topic?\n\nI personnelly liek the idea of a billboard in space. But problem. How do you\nservice it? fly a shuttle/DC-1 to near it and then dismount and ""fly"" to it?\nOr what?? or havign a special docking section for shuttle/DC-1 docking?\n\nAlso what if the billboard springs a leak? Self sealing and such??\n\n\nJust thinking (okay rambling)..\n\nAlso why must the now inflated billboard, not be covered in the inside by a\nharder substance (such as a polymer or other agent) and then the now ""hard""\nbillboard would be a now giant docking structure/space dock/station??",14,sci.space
1616,"\n Just because the 68070 can run upto 15Mhz doesn't mean the CD-I\nis running at that speed. I said -> I understand it is a 68070 running\nat something like 7Mhz. I am not sure, but I think I read this a long\ntime ago.\n\n Anyway, still with 15Mhz, you need sprites for a lot of tricks for\nmaking cool awesome games (read psygnosis).\n",1,comp.graphics


In [28]:
# For reference on regex: https://docs.python.org/3/library/re.html

# From 'content' column: 

# 1.. Remove new line characters
df['clean_text']=df['content'].apply(lambda x: re.sub('\s+', ' ', x)) #includes /n.. look into 

# 2.. Remove Emails
df['clean_text']=df['clean_text'].apply(lambda x: re.sub('From: \S+@\S+', '', x))

# 3. Remove non-alphanumeric characters
df['clean_text']=df['clean_text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))



# 4.. Remove extra whitespace 
df['clean_text']=df['clean_text'].apply(lambda x: ' '.join(x.split()))


In [29]:
df.sample(3)

,content,target,target_names,clean_text
5988,"^^^^^\n\nWhat an incrediblt sexist remark! Come now, Mike, what ever possessed you to\nmake such a un-PC remark? I hope all women out there reading this are as\nincensed as I am. Remember, WOMAN ARE JUST AS GOOD AS MEN!!!! \n\nWomen stand up for your right to be just as stupid as men. In fact, insist on\nevery oppurtunity to be even more stupid than men! You've got the right, use\nit!\n\nHey, it's a slow afternoon and I really don't want to get back to that\nreport...;)\n\nBTW: mega-smileys for the humor impaired...",16,talk.politics.guns,What an incrediblt sexist remark Come now Mike what ever possessed you to make such a un PC remark I hope all women out there reading this are as incensed as I am Remember WOMAN ARE JUST AS GOOD AS MEN Women stand up for your right to be just as stupid as men In fact insist on every oppurtunity to be even more stupid than men You ve got the right use it Hey it s a slow afternoon and I really don t want to get back to that report BTW mega smileys for the humor impaired
9444,Diamond engagement ring. 14k gold band. 33point diamond. appraised at\n1900 dollars. Will sell for 600 dollars. Appraisal available upon request.\nsend e-mail to yb025@uafhp.uark.edu\n,6,misc.forsale,Diamond engagement ring k gold band point diamond appraised at dollars Will sell for dollars Appraisal available upon request send e mail to yb uafhp uark edu
7217,"Avi,\n For your information, Islam permits freedom of religion - there is\nno compulsion in religion. Does Judaism permit freedom of religion\n(i.e. are non-Jews recognized in Judaism). Just wondering.",17,talk.politics.mideast,Avi For your information Islam permits freedom of religion there is no compulsion in religion Does Judaism permit freedom of religion i e are non Jews recognized in Judaism Just wondering


In [31]:
nlp = spacy.load("en_core_web_lg")

In [32]:
# Leverage tqdm for progress_apply
from tqdm import tqdm
tqdm.pandas()

# If you're on macOS, Linux, or python session executed from Windows Subsystem for Linux (WSL)
# conda activate U4-S1-NLP
# pip install pandarallel
#
# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)
#
# df['lemmas'] = df['content'].parallel_apply(get_lemmas)
#
# Ref: https://github.com/nalepae/pandarallel

In [34]:
# Create 'lemmas' column
def get_lemmas(x):
    lemmas = []
    for token in nlp(x):
        if (token.is_stop!=True) and (token.is_punct!=True):
            lemmas.append(token.lemma_)
    return lemmas

df['lemmas'] = df['clean_text'].progress_apply(get_lemmas)


  3%|█                                      | 310/11314 [00:25<17:39, 10.39it/s]


  5%|██                                     | 597/11314 [00:52<15:04, 11.85it/s]


  8%|███                                    | 895/11314 [01:12<21:48,  7.96it/s]


 10%|███▊                                  | 1134/11314 [01:35<55:46,  3.04it/s]


 13%|████▊                                 | 1450/11314 [01:56<15:21, 10.70it/s]


...

 93%|██████████████████████████████████▌  | 10570/11314 [17:29<00:47, 15.57it/s]


 96%|███████████████████████████████████▍ | 10842/11314 [17:56<02:05,  3.76it/s]


 98%|████████████████████████████████████▏| 11048/11314 [18:17<00:18, 14.14it/s]


 99%|████████████████████████████████████▋| 11228/11314 [18:41<00:12,  6.64it/s]


100%|█████████████████████████████████████| 11314/11314 [18:54<00:00,  9.98it/s]


In [36]:
df.head()

,content,target,target_names,clean_text,lemmas
0,"I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.",7,rec.autos,I was wondering if anyone out there could enlighten me on this car I saw the other day It was a door sports car looked to be from the late s early s It was called a Bricklin The doors were really small In addition the front bumper was separate from the rest of the body This is all I know If anyone can tellme a model name engine specs years of production where this car is made history or whatever info you have on this funky looking car please e mail,"[wonder, enlighten, car, see, day, , , door, sport, car, , look, late, , s, , early, , s, , call, Bricklin, , door, small, , addition, , bumper, separate, rest, body, , know, , tellme, model, , engine, spec, , year, production, , car, , history, , info, funky, look, car, , e, mail]"
1,"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.",4,comp.sys.mac.hardware,A fair number of brave souls who upgraded their SI clock oscillator have shared their experiences for this poll Please send a brief message detailing your experiences with the procedure Top speed attained CPU rated speed add on cards and adapters heat sinks hour of usage per day floppy disk functionality with and m floppies are especially requested I will be summarizing in the next two days so please add to the network knowledge base if you have done the clock upgrade and haven t answered this poll Thanks,"[fair, number, brave, soul, upgrade, SI, clock, oscillator, share, experience, poll, , send, brief, message, detail, experience, procedure, , speed, attain, , cpu, rate, speed, , add, card, adapter, , heat, sink, , hour, usage, day, , floppy, disk, functionality, , , m, floppy, especially, request, , summarize, day, , add, network, knowledge, base, clock, upgrade, haven, t, answer, poll, , thank]"
2,"well folks, my mac plus finally gave up the ghost this weekend after\nstarting life as a 512k way back in 1985. sooo, i'm in the market for a\nnew machine a bit sooner than i intended to be...\n\ni'm looking into picking up a powerbook 160 or maybe 180 and have a bunch\nof questions that (hopefully) somebody can answer:\n\n* does anybody know any dirt on when the next round of powerbook\nintroductions are expected? i'd heard the 185c was supposed to make an\nappearence ""this summer"" but haven't heard anymore on it - and since i\ndon't have access to macleak, i was wondering if anybody out there had\nmore info...\n\n* has anybody heard rumors about price drops to the powerbook line like the\nones the duo's just went through recently?\n\n* what's the impression of the display on the 180? i could probably swing\na 180 if i got the 80Mb disk rather than the 120, but i don't really have\na feel for how much ""better"" the display is (yea, it looks great in the\nstore, but is that all ""wow"" or is it really that good?). could i solicit\nsome opinions of people who use the 160 and 180 day-to-day on if its worth\ntaking the disk size and money hit to get the active display? (i realize\nthis is a real subjective question

### The two main inputs to the LDA topic model are the dictionary (id2word) and the corpus.

In [37]:
# Create Dictionary
id2word = corpora.Dictionary(df['lemmas'] )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in df['lemmas']]

In [38]:
# How many words do we have?
len(id2word.keys())

77754

In [39]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.75)

In [40]:
# How many words do we have?
len(id2word.keys())

14778

In [48]:
id2word[249]

'prob'

In [53]:
df['content'][4]

"From article <C5owCB.n3p@world.std.com>, by tombaker@world.std.com (Tom A Baker):\n\n\nMy understanding is that the 'expected errors' are basically\nknown bugs in the warning system software - things are checked\nthat don't have the right values in yet because they aren't\nset till after launch, and suchlike. Rather than fix the code\nand possibly introduce new bugs, they just tell the crew\n'ok, if you see a warning no. 213 before liftoff, ignore it'."

In [43]:
corpus[5]

[(0, 11),
 (117, 1),
 (177, 1),
 (193, 1),
 (221, 1),
 (225, 1),
 (226, 1),
 (227, 1),
 (228, 1),
 (229, 1),
 (230, 1),
 (231, 1),
 (232, 1),
 (233, 1),
 (234, 1),
 (235, 1),
 (236, 1),
 (237, 1),
 (238, 1),
 (239, 1),
 (240, 1),
 (241, 1),
 (242, 1),
 (243, 1),
 (244, 1),
 (245, 1),
 (246, 2),
 (247, 1),
 (248, 1),
 (249, 2)]

In [44]:
id2word[252]

'rm'

In [45]:
id2word[276]

'controller'

In [46]:
# Human readable format of corpus (term-frequency)
[(id2word[word_id], word_count) for word_id, word_count in corpus[5]]

[('  ', 11),
 ('helpful', 1),
 ('address', 1),
 ('ignore', 1),
 ('course', 1),
 ('evidently', 1),
 ('later', 1),
 ('mass', 1),
 ('point', 1),
 ('present', 1),
 ('quote', 1),
 ('read', 1),
 ('switch', 1),
 ('term', 1),
 ('topic', 1),
 ('understand', 1),
 ('weapon', 1),
 ('News', 1),
 ('Sean', 1),
 ('September', 1),
 ('Sharon', 1),
 ('accidentally', 1),
 ('bounce', 1),
 ('couldn', 1),
 ('delete', 1),
 ('directly', 1),
 ('file', 2),
 ('glad', 1),
 ('instead', 1),
 ('prob', 2)]

# Part 2: Estimate a LDA Model with Gensim

 ### Train an LDA model

In [51]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20, 
                                            chunksize=100,
                                            passes=10,
                                            per_word_topics=True)

# https://radimrehurek.com/gensim/models/ldamodel.html

IndexError: index 14778 is out of bounds for axis 1 with size 14778

In [50]:
lda_model.save('lda_model.model')

NameError: name 'lda_model' is not defined

In [ ]:
%%time
lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics=20, 
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=12)

# https://radimrehurek.com/gensim/models/ldamulticore.html

In [ ]:
lda_multicore.save('lda_multicore.model')

In [ ]:
from gensim import models
lda_multicore =  models.LdaModel.load('lda_multicore.model')

### View the topics in LDA model

In [ ]:
newsgroups_train.target_names

In [ ]:
pprint(lda_multicore.print_topics())
doc_lda = lda_multicore[corpus]

In [ ]:
lda_multicore[corpus[5]][0]

In [ ]:
doc_lda

In [ ]:
distro = [lda[d] for d in corpus]

In [ ]:
#distro = [lda_multicore[corpus[d]][0] for d in corpus]

In [ ]:
distro[0]

### What is topic Perplexity?
Perplexity is a statistical measure of how well a probability model predicts a sample. As applied to LDA, for a given value of , you estimate the LDA model. Then given the theoretical word distributions represented by the topics, compare that to the actual topic mixtures, or distribution of words in your documents.

### What is topic coherence?
Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference.
A set of statements or facts is said to be coherent, if they support each other. Thus, a coherent fact set can be interpreted in a context that covers all or most of the facts. An example of a coherent fact set is “the game is a team sport”, “the game is played with a ball”, “the game demands great physical efforts”

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_multicore.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_multicore, 
                                     texts=df['lemmas'], 
                                     dictionary=id2word, 
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Part 3: Interpret LDA results & Select the appropriate number of topics

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_multicore, corpus, id2word)
pyLDAvis.display(vis)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics=num_topics, 
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=12)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
%%time
model_list, coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        texts=df['lemmas'], 
                                                        start=2, 
                                                        limit=40, 
                                                        step=6)

In [ ]:
coherence_values = [0.5054, 0.5332, 0.5452, 0.564, 0.5678, 0.5518, 0.519]

In [ ]:
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
#optimal_model = model_list[4]
optimal_model =  models.LdaModel.load('optimal_model.model')
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))